In [1]:
import pandas as pd 
import numpy as np
import sqlite3
conn = sqlite3.connect('CourseDataWareHouse.db')
c = conn.cursor()

In [2]:
%load_ext sql

In [3]:
%%sql
sqlite:///CourseDataWareHouse.db

'Connected: @CourseDataWareHouse.db'

In [82]:
%%sql
DROP TABLE IF EXISTS COURSES;
DROP TABLE IF EXISTS MEETINGS;
DROP TABLE IF EXISTS PROFESSOR;
DROP TABLE IF EXISTS TIME;
DROP TABLE IF EXISTS COURSE_FACTS;

 * sqlite:///CourseData.db
   sqlite:///CourseDataWareHouse.db
Done.
Done.
Done.
Done.
Done.


[]

In [83]:
%%sql
create table 'COURSES' (
    CO_ID INTEGER PRIMARY KEY,
    PROGRAM_CODE TEXT NOT NULL,
    PROGRAM_NAME VARCHAR NOT NULL,
    COURSE_TITLE VARCHAR NOT NULL, 
    CREDITS VARCHAR NOT NULL,
    PRE_REQS VARCHAR, 
    CO_REQS VARCHAR
);

 * sqlite:///CourseData.db
   sqlite:///CourseDataWareHouse.db
Done.


[]

In [6]:
%%sql
create table 'MEETINGS' (
    ROOM_ID INTEGER PRIMARY KEY,
    ROOM_NO VARCHAR NOT NULL,
    BUILDING VARCHAR,
    STREET VARCHAR,
    AREACODE INTEGER
);

 * sqlite:///CourseDataWareHouse.db
Done.


[]

In [44]:
%%sql
create table 'PROFESSOR'(
    I_ID INTEGER PRIMARY KEY,
    NAME VARCHAR NOT NULL,
    PROGRAM_NAME VARCHAR NOT NULL,
    COURSE_TITLE VARCHAR NOT NULL,
    ROOM_NO VARCHAR NOT NULL
);


 * sqlite:///CourseData.db
   sqlite:///CourseDataWareHouse.db
Done.


[]

In [8]:
%%sql
create table 'TIME'(
    TIME_ID INTEGER PRIMARY KEY,
    DAY VARCHAR,
    TERM VARCHAR NOT NULL
);

 * sqlite:///CourseDataWareHouse.db
Done.


[]

In [9]:
%%sql
create table 'COURSE_FACTS'(
    PROGRAM_CODE VARCHAR NOT NULL,
    CM_ID INTEGER NOT NULL,
    TERM_ID INTEGER NOT NULL,
    I_ID INTEGER NOT NULL,
    TIME_ID INTEGER NOT NULL,
    ROOM_ID INTEGER NOT NULL,
    CAP INTEGER NOT NULL,
    ACT INTEGER,
    REM INTEGER,
    MAX_CREDITS INTEGER NOT NULL,
    MAX CAP INTEGER NOT NULL,
    MIN REM INTEGER,
    FOREIGN KEY (PROGRAM_CODE) REFERENCES COURSES (PROGRAM_CODE)
    FOREIGN KEY (ROOM_ID) REFERENCES MEETINGS (ROOM_ID)
    FOREIGN KEY (I_ID) REFERENCES INSTRUCTOR (I_ID)
    FOREIGN KEY (TIME_ID) REFERENCES TIME (TIME_ID)
);

 * sqlite:///CourseDataWareHouse.db
Done.


[]

In [10]:
%%sql
sqlite:///CourseData.db

'Connected: @CourseData.db'

In [11]:
conn_new = sqlite3.connect('CourseData.db')
c_1 = conn.cursor()

In [88]:
%%sql
INSERT INTO COURSES (CO_ID, PROGRAM_CODE, PROGRAM_NAME, COURSE_TITLE, CREDITS, PRE_REQS, CO_REQS)
SELECT DISTINCT c.co_id, a.PROGRAM_CODE, a.PROGRAM_NAME, a.COURSE_TITLE, c.CREDITS, b.PRE_REQS, b.CO_REQS
FROM CATALOG_PROGRAM a, CATALOG_COURSE b, COURSE_OFFERING c
where a.catalog_id=b.catalog_id
and b.course_title=c.course_title;

 * sqlite:///CourseData.db
   sqlite:///CourseDataWareHouse.db


IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: COURSES.CO_ID [SQL: 'INSERT INTO COURSES (CO_ID, PROGRAM_CODE, PROGRAM_NAME, COURSE_TITLE, CREDITS, PRE_REQS, CO_REQS)\nSELECT DISTINCT c.co_id, a.PROGRAM_CODE, a.PROGRAM_NAME, a.COURSE_TITLE, c.CREDITS, b.PRE_REQS, b.CO_REQS\nFROM CATALOG_PROGRAM a, CATALOG_COURSE b, COURSE_OFFERING c\nwhere a.catalog_id=b.catalog_id\nand b.course_title=c.course_title;'] (Background on this error at: http://sqlalche.me/e/gkpj)

In [87]:
%%sql
SELECT DISTINCT c.co_id, a.PROGRAM_CODE, a.PROGRAM_NAME, a.COURSE_TITLE, c.CREDITS, b.PRE_REQS, b.CO_REQS
FROM CATALOG_PROGRAM a, CATALOG_COURSE b, COURSE_OFFERING c
where a.catalog_id=b.catalog_id
and b.course_title=c.course_title
limit 1;

 * sqlite:///CourseData.db
   sqlite:///CourseDataWareHouse.db
Done.


CO_ID,PROGRAM_CODE,PROGRAM_NAME,COURSE_TITLE,CREDITS,PRE_REQS,CO_REQS
6949,AN,Asian Studies,Independent Study,0.000-3.000,None,None


In [ ]:
%%sql
INSERT INTO COURSE_OFFERING_W (ACT, CAP, REM, FID, LID, TERMID, CID)
SELECT DISTINCT ACT, CAP, REM, FID, LID,TERMID, CID
FROM COURSE_OFFERING
    LEFT JOIN FACULTY_W USING (FID)
    LEFT JOIN CLASS_MEETINGS USING (COID)
    LEFT JOIN LOCATION USING (LID)
    JOIN TERM_W USING (SEMESTER)
    JOIN COURSE_W USING (CID)

In [ ]:
import pandas as pd

In [25]:
programs = 'SELECT PROGRAM_CODE, PROGRAM_NAME, COURSE_TITLE FROM CATALOG_PROGRAM;'

In [26]:
programs_df = pd.read_sql(programs, conn_new)
programs_df.head()

,PROGRAM_CODE,PROGRAM_NAME,COURSE_TITLE
0,AN,Asian Studies,Independent Study
1,AN,Asian Studies,Asian Studies Seminar
2,BU,Business,Legal Environment of Business
3,BU,Business,Environmental Law and Policy
4,BU,Business,"The Law of Contracts, Sales, and Property"


In [34]:
COURSE_DETAILS = 'SELECT PRE_REQS, CO_REQS FROM CATALOG_COURSE;'

In [35]:
courses_details_df = pd.read_sql(COURSE_DETAILS, conn_new)
courses_details_df.head()

,PRE_REQS,CO_REQS
0,None,None
1,None,None
2,Junior standing.,None
3,None,None
4,BU 0211.,None


In [31]:
credits = 'SELECT CREDITS FROM COURSE_OFFERING;'

In [33]:
credits_df = pd.read_sql(credits, conn_new)
credits_df.head()

,CREDITS
0,3.000
1,3.000
2,3.000
3,3.000
4,3.000


In [58]:
df_courses = pd.concat([programs_df,courses_details_df,credits_df], axis=1)
df_courses.head()

,PROGRAM_CODE,PROGRAM_NAME,COURSE_TITLE,PRE_REQS,CO_REQS,CREDITS
0,AN,Asian Studies,Independent Study,None,None,3.000
1,AN,Asian Studies,Asian Studies Seminar,None,None,3.000
2,BU,Business,Legal Environment of Business,Junior standing.,None,3.000
3,BU,Business,Environmental Law and Policy,None,None,3.000
4,BU,Business,"The Law of Contracts, Sales, and Property",BU 0211.,None,3.000


In [67]:
df_courses.to_sql('COURSES', conn_new, if_exists='append', index=False)

IntegrityError: UNIQUE constraint failed: COURSES.PROGRAM_CODE

In [47]:
%%sql 
select * from COURSES

 * sqlite:///CourseData.db
   sqlite:///CourseDataWareHouse.db
Done.


program_code,program_name,COURSE_TITLE,credits,prereqs,coreqs


In [14]:
courses = 'SELECT * FROM COURSE_OFFERING;'

In [15]:
courses_df = pd.read_sql(courses, conn_new)
courses_df.head()

,CO_ID,CRN,SECTION,COURSE_TITLE,CREDITS
0,1,70384,C01,Introduction to Financial Accounting,3.000
1,2,70385,C02,Introduction to Financial Accounting,3.000
2,3,70382,C03,Introduction to Financial Accounting,3.000
3,4,70291,C04,Introduction to Financial Accounting,3.000
4,5,70350,C05,Introduction to Financial Accounting,3.000


In [16]:
meetings = 'SELECT * FROM LOCATION';

In [17]:
meetings_df = pd.read_sql(meetings, conn_new)
meetings_df.head()

,ROOM_ID,Room_No,CRN
0,1,DSB 105,70384
1,2,DSB 105,70385
2,3,DSB 105,70382
3,4,DSB 111,70291
4,5,DSB 111,70350


In [18]:
instructor = 'SELECT NAME, COURSE_TITLE, CRN FROM INSTRUCTOR'

In [19]:
instructor_df = pd.read_sql(instructor, conn_new)
instructor_df.head()

,NAME,COURSE_TITLE,CRN
0,Michael P. Coyne,Introduction to Financial Accounting,70384
1,Michael P. Coyne,Introduction to Financial Accounting,70385
2,Michael P. Coyne,Introduction to Financial Accounting,70382
3,Rebecca I. Bloch,Introduction to Financial Accounting,70291
4,Rebecca I. Bloch,Introduction to Financial Accounting,70350


In [21]:
df1 = pd.concat([courses_df,instructor_df,], axis=1)
df1.head()

,CO_ID,CRN,SECTION,COURSE_TITLE,CREDITS,NAME,COURSE_TITLE,CRN
0,1,70384,C01,Introduction to Financial Accounting,3.000,Michael P. Coyne,Introduction to Financial Accounting,70384.0
1,2,70385,C02,Introduction to Financial Accounting,3.000,Michael P. Coyne,Introduction to Financial Accounting,70385.0
2,3,70382,C03,Introduction to Financial Accounting,3.000,Michael P. Coyne,Introduction to Financial Accounting,70382.0
3,4,70291,C04,Introduction to Financial Accounting,3.000,Rebecca I. Bloch,Introduction to Financial Accounting,70291.0
4,5,70350,C05,Introduction to Financial Accounting,3.000,Rebecca I. Bloch,Introduction to Financial Accounting,70350.0


In [ ]:
df1.to_sql('courses', conn, if_exists='append', index=False)

In [36]:
course = instructor_df ['COURSE_TITLE'] == 'Introduction to Financial Accounting'
instructor_df[course]

,COURSE_TITLE,CRN
NAME,,
Michael P. Coyne,Introduction to Financial Accounting,70384
Michael P. Coyne,Introduction to Financial Accounting,70385
Michael P. Coyne,Introduction to Financial Accounting,70382
Rebecca I. Bloch,Introduction to Financial Accounting,70291
Rebecca I. Bloch,Introduction to Financial Accounting,70350
Rebecca I. Bloch,Introduction to Financial Accounting,70381
Paul Caster,Introduction to Financial Accounting,70383
Jo Ann Drusbosky,Introduction to Financial Accounting,70391
Jo Ann Drusbosky,Introduction to Financial Accounting,71105


In [40]:
time = 'SELECT * FROM TIME_CODE';

In [41]:
time_df = pd.read_sql(time, conn_new)
time_df.head()

,TIME_ID,START_TIME,END_TIME,CRN
0,1,2014-09-02T08:00:00,2014-09-02T09:15:00,70384
1,2,2014-09-05T08:00:00,2014-09-05T09:15:00,70384
2,3,2014-09-09T08:00:00,2014-09-09T09:15:00,70384
3,4,2014-09-12T08:00:00,2014-09-12T09:15:00,70384
4,5,2014-09-16T08:00:00,2014-09-16T09:15:00,70384


In [42]:
time_df['START_TIME']=pd.to_datetime(time_df['START_TIME'])
time_df['START_TIME'].head()

0   2014-09-02 08:00:00
1   2014-09-05 08:00:00
2   2014-09-09 08:00:00
3   2014-09-12 08:00:00
4   2014-09-16 08:00:00
Name: START_TIME, dtype: datetime64[ns]

In [43]:
time_df['END_TIME']=pd.to_datetime(time_df['END_TIME'])
time_df['END_TIME'].head()

0   2014-09-02 09:15:00
1   2014-09-05 09:15:00
2   2014-09-09 09:15:00
3   2014-09-12 09:15:00
4   2014-09-16 09:15:00
Name: END_TIME, dtype: datetime64[ns]

In [39]:
Avg_Act = meetings_df['ACT'].mean()
print(Avg_Act)

KeyError: 'ACT'